In [ ]:
import pandas as pd
import pandas_ta as ta
from datetime import datetime

In [ ]:


# Charger tes données dans un DataFrame
df = pd.read_csv('../DB/bitget/1h/BTC-USDT-USDT.csv') 
df['date'] = df['date'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
df.set_index('date', inplace=True)
df[:5]



In [ ]:
df_test = df['2024-07-01 00:00:00':] 
df_test

In [ ]:
# Fonction pour calculer le Parabolic SAR
def calculate_parabolic_sar(df, initial_af=0.02, max_af=0.2):
    sar = df['close'].copy()
    sar[0] = df['close'][0]
    ep = df['high'][0]
    af = initial_af
    trend = 1  # 1 = bull, -1 = bear

    for i in range(1, len(df)):
        sar[i] = sar[i-1] + af * (ep - sar[i-1])

        if trend == 1:
            if df['low'][i] < sar[i]:
                trend = -1
                sar[i] = ep
                ep = df['low'][i]
                af = initial_af
            else:
                if df['high'][i] > ep:
                    ep = df['high'][i]
                    af = min(max_af, af + initial_af)
        else:
            if df['high'][i] > sar[i]:
                trend = 1
                sar[i] = ep
                ep = df['high'][i]
                af = initial_af
            else:
                if df['low'][i] < ep:
                    ep = df['low'][i]
                    af = min(max_af, af + initial_af)

    return sar


df_test['SAR'] = calculate_parabolic_sar(df_test)

In [ ]:
df_test

# Calcul des signaux d'achat et de vente
df_test['signal'] = 0
df_test['signal'][df_test['close'] > df_test['SAR']] = 1   # Signal d'achat
df_test['signal'][df_test['close'] < df_test['SAR']] = -1  # Signal de vente

df_test

In [ ]:

import pandas as pd
import plotly.graph_objects as go

In [ ]:

def plot_chart(df):

    # Création du graphique
    fig = go.Figure()

    # Ajouter les bougies
    fig.add_trace(go.Candlestick(
        x=df.index,
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name='Candlestick',
        line=dict(width=1), opacity=1
    ))

    # Ajouter le Parabolic SAR
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['SAR'],
        mode='lines',
        name='Parabolic SAR',
        line=dict(color='orange')
    ))

    # Ajouter les signaux d'achat et de vente
    fig.add_trace(go.Scatter(
        x=df.index[df['signal'] == 1],
        y=df['close'][df['signal'] == 1],
        mode='markers',
        marker=dict(color='green', symbol='triangle-up', size=10),
        name='Buy Signal'
    ))

    fig.add_trace(go.Scatter(
        x=df.index[df['signal'] == -1],
        y=df['close'][df['signal'] == -1],
        mode='markers',
        marker=dict(color='red', symbol='triangle-down', size=10),
        name='Sell Signal'
    ))

    # Mettre en forme le graphique
    fig.update_layout(
        title='Parabolic SAR Strategy',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False,
        height=600
    )

    # Afficher le graphique
    return fig.show()

plot_chart(df_test)

In [ ]:
def populate_positions(df):

    # Simuler les trades
    df['position'] = df['signal'].shift()  # Position du jour précédent
    df['trade_return'] = df['close'].pct_change() * df['position']  # Retour de chaque trade

    # Calculer le retour cumulé
    df['cumulative_return'] = (1 + df['trade_return']).cumprod() - 1

    # Calculer les profits
    df['profit'] = df['trade_return'] * 100  # Pourcentage de profit

    # Résumé des profits
    total_profit = df['profit'].sum()
    total_return = df['cumulative_return'].iloc[-1] * 100  # Retour total en pourcentage

    print(f"Total Profit: {total_profit:.2f}%")
    print(f"Total Return: {total_return:.2f}%")

    # Afficher le DataFrame avec les profits
    print(df[['close', 'SAR', 'signal', 'position', 'trade_return', 'cumulative_return', 'profit']])

populate_positions(df_test)    
